In [ ]:
import numpy as np
import torch
import meshplot as mp # useful for visualizing
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import primitive
import trimesh

In [ ]:
# Loading a mesh and do uniformly samplings on the surface

# mesh = trimesh.load_mesh('segmentedRod.obj')
mesh = trimesh.load_mesh('rod/segmentedRodCable_segbyUI.obj')

# Sample points uniformly from the surface of the mesh
points, _ = trimesh.sample.sample_surface_even(mesh, 1000)  # sample normally? 
points_numpy = np.array(points)
points_tensor = torch.tensor(points_numpy, dtype=torch.float32)


In [ ]:
# # diff. learning on a certain mesh input

# import pytorch3d 
# from pytorch3d.loss import chamfer_distance

# # normal distribution
# opt_params_list = []
# for i in range(5):
#     opt_params = torch.randn(8)
#     opt_params[-2:] = torch.abs(opt_params[-2:])
#     opt_params.requires_grad = True

#     optimizer = torch.optim.Adam([opt_params], 0.1) # 0.1 is a good lr?
#     uv_coordinates = torch.rand((1000,2))
#     losses = []

#     points_tensor = points_tensor.view(1, -1, 3) 

#     # If the gragh doesn't look good, we need to rerun it.

#     for _ in tqdm(range(5000)): # for the progress bar
#         optimizer.zero_grad()
#         opt_points = primitive.cylinder(opt_params, uv_coordinates)
#         opt_points = opt_points.view(1, -1, 3)

#         loss = chamfer_distance(x = points_tensor, y = opt_points)
#         loss = loss[0]
        
#         loss.backward()
#         optimizer.step()

#         with torch.no_grad():  # Perform operations without tracking gradients
#             opt_params[-2:] = torch.abs(opt_params[-2:])  # Ensure last two elements are positive
#             if opt_params[-1] <= opt_params[-2]:
#                 opt_params[-1] = opt_params[-2] + 0.1  # Ensure last element is always greater than the second last

            
#         losses.append(loss.detach().item())
#     plt.plot(losses)
#     plt.show
#     opt_params_list.append(opt_params.detach().numpy())

In [ ]:
# diff. learning on a certain mesh input

import pytorch3d 
from pytorch3d.loss import chamfer_distance

# normal distribution

opt_params = torch.randn(8)
opt_params[-2:] = torch.abs(opt_params[-2:])
opt_params.requires_grad = True

optimizer = torch.optim.Adam([opt_params], 0.1) # 0.1 is a good lr?
uv_coordinates = torch.rand((1000,2))
losses = []

points_tensor = points_tensor.view(1, -1, 3) 

# If the gragh doesn't look good, we need to rerun it.

for _ in tqdm(range(5000)): # for the progress bar
    optimizer.zero_grad()
    opt_points = primitive.cylinder(opt_params, uv_coordinates)
    opt_points = opt_points.view(1, -1, 3)

    loss = chamfer_distance(x = points_tensor, y = opt_points)
    loss = loss[0]
    
    loss.backward()
    optimizer.step()

    with torch.no_grad():  # Perform operations without tracking gradients
        opt_params[-2:] = torch.abs(opt_params[-2:])  # Ensure radius and length are positive
        if opt_params[-1] <= opt_params[-2]:
            opt_params[-1] = opt_params[-2] + 0.1  # Ensure length is always greater than the radius

    losses.append(loss.detach().item())
plt.plot(losses)
plt.show


In [ ]:
print(np.mean(opt_params_list, axis=0))
print(opt_params_list)
print(np.std(np.stack(opt_params_list), axis=0))

In [ ]:
# compare the original mesh with the optimized mesh
points_tensor = points_tensor.reshape(-1, 1, 3)
plot = mp.plot(points_tensor.detach().numpy(), return_plot=True, shading={'point_size':2})

opt_points = opt_points.reshape(-1, 1, 3)
plot.add_points(opt_points.detach().numpy(), shading={'point_size':2, 'point_color':'green'})
print(opt_params)

In [ ]:
# check opt_params by plotting original mesh with the points cloud got by opt_params
# orignial mesh points cloud
plot = mp.plot(points_tensor.detach().numpy(), return_plot=True, shading={'point_size':2})

# optimized mesh points cloud
coordinates = torch.rand((1000, 2))
# opt_params is just the final parameters, so we have center, position, radius and length stored in opt_params
# opt_params.detach().numpy()[-1] = 68
points = primitive.cylinder(opt_params, coordinates).detach().numpy()
plot.add_points(points, shading={'point_size':2, 'point_color':'green'})
print(opt_params)

In [ ]:
# Loading a mesh and do uniformly samplings on the surface
mesh = trimesh.load_mesh('rod/segmentedRodCable_segbyUI2.obj')

# Sample points uniformly from the surface of the mesh
points2, _ = trimesh.sample.sample_surface_even(mesh, 1000)
points_numpy2 = np.array(points2)
points_tensor2 = torch.tensor(points_numpy2, dtype=torch.float32)


In [ ]:
# plot = mp.plot(points_tensor2.reshape(-1, 1, 3).detach().numpy(), return_plot=True, shading={'point_size':1})

# coordinates2 = torch.rand((1000, 2))
# points2 = primitive.cylinder(torch.tensor([-4.1692504e-03,  2.8632283e-02, -2.0405367e+01, 0, 0, 1,  2.3550220e+00,  1.1063369e+01]), coordinates2).detach().numpy()
# plot.add_points(points2, shading={'point_size':1, 'point_color':'green'})

In [ ]:
# diff. learning on a certain mesh input

import pytorch3d 
from pytorch3d.loss import chamfer_distance

opt_params_list2 = []
for i in range(5):
    opt_params2 = torch.randn(8)
    opt_params2[-2:] = torch.abs(opt_params2[-2:])
    opt_params2.requires_grad = True

    optimizer2 = torch.optim.Adam([opt_params2], 0.1) # 1 is a good lr
    uv_coordinates2 = torch.rand((1000,2))
    losses2 = []

    points_tensor2 = points_tensor2.view(1, -1, 3) 


    for _ in tqdm(range(5000)): # for the progress bar
        optimizer2.zero_grad()
        opt_points2 = primitive.cylinder(opt_params2, uv_coordinates2)
        opt_points2 = opt_points2.view(1, -1, 3)

        loss2 = chamfer_distance(x = points_tensor2, y = opt_points2) 
        loss2 = loss2[0]
        
        loss2.backward()
        optimizer2.step()

        with torch.no_grad():
            opt_params2[-2:] = torch.abs(opt_params2[-2:]) 
            if opt_params2[-1] <= opt_params2[-2]:
                opt_params2[-1] = opt_params2[-2] + 0.01

        losses2.append(loss2.detach().item())
    plt.plot(losses2)
    plt.show
    opt_params_list2.append(opt_params2.detach().numpy())

In [ ]:
print(np.mean(opt_params_list2, axis=0))
print(opt_params_list2)
print(np.std(np.stack(opt_params_list2), axis=0))

In [ ]:
# same, compare the original mesh with the optimized mesh
points_tensor2 = points_tensor2.reshape(-1, 1, 3)
plot = mp.plot(points_tensor2.detach().numpy(), return_plot=True, shading={'point_size':2})

opt_points2 = opt_points2.reshape(-1, 1, 3)
plot.add_points(opt_points2.detach().numpy(), shading={'point_size':2, 'point_color':'green'})
print(opt_params2)

In [ ]:
# check opt_params by plotting original mesh with the points cloud got by opt_params
# orignial mesh points cloud
plot2 = mp.plot(points_tensor2.detach().numpy(), return_plot=True, shading={'point_size':2})

# optimized mesh points cloud
coordinates2 = torch.rand((1000, 2))
# opt_params2.detach().numpy()[-1] = 50
points2 = primitive.cylinder(opt_params2, coordinates2).detach().numpy()
plot2.add_points(points2, shading={'point_size':2, 'point_color':'green'})
print(opt_params2)

In [ ]:
# so we have 3 types of points cloud, the original, the optimzied one, the one got by plugging in optimized parameters

In [ ]:
# red is the the long, original rod
plot = mp.plot(points_tensor.detach().numpy(), return_plot=True, shading={'point_size':5})
# green is the long, optimized rod
plot.add_points(opt_points.detach().numpy(), shading={'point_size':5, 'point_color':'green'})
# black is the short, orignial rod
plot.add_points(points_tensor2.detach().numpy(), shading={'point_size':5, 'point_color':'black'})
# purple is the short optimized rod
plot.add_points(opt_points2.detach().numpy(), shading={'point_size':5, 'point_color':'purple'})

print(opt_params)
print(opt_params2)

In [ ]:
opt_params = opt_params.detach().numpy()
opt_params2 = opt_params2.detach().numpy()

print(opt_params)  # optimized long rod parameters
print(opt_params2)  # optimized short rod parameters

In [ ]:
def compute_l2_distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

shape_param1 = opt_params[-2:]
shape_param2 = opt_params2[-2:]
print(compute_l2_distance(shape_param1, shape_param2))

position_param1 = opt_params[:3]
position_param2 = opt_params2[:3]
print(compute_l2_distance(position_param1, position_param2))

orientation_param1 = opt_params[3:6]
orientation_param2 = opt_params2[3:6]
print(compute_l2_distance(orientation_param1, orientation_param2))